In [1]:
import pandas as pd
from helpers import column_is_constant

In [2]:
staffing = pd.read_csv("output/staffing-with-fiscal.csv")
staffing.head()

,City,report_year,index,Assignment Category,Educator Type,FTE Count,OrganizationCode,OrganizationType,Education spending,Enrollment
0,NEW HAVEN,2009,Achievement First Hartford Academy Inc. District,Administrators Coordinators and Department Ch...,Certified,6.0,2880013,Public Charter School Districts,192024547.0,18074.31
1,NEW HAVEN,2009,Achievement First Hartford Academy Inc. District,General Education - Teachers and Instructors,Certified,20.0,2880013,Public Charter School Districts,192024547.0,18074.31
2,NEW HAVEN,2009,Achievement First Hartford Academy Inc. District,General Education - Paraprofessional Instructi...,Non-Certified,0.0,2880013,Public Charter School Districts,192024547.0,18074.31
3,NEW HAVEN,2009,Achievement First Hartford Academy Inc. District,Library/Media - Support Staff,Non-Certified,0.0,2880013,Public Charter School Districts,192024547.0,18074.31
4,NEW HAVEN,2009,Achievement First Hartford Academy Inc. District,School Nurses,Non-Certified,0.5,2880013,Public Charter School Districts,192024547.0,18074.31


In [3]:
pubs = staffing[staffing["OrganizationType"] == "Public School Districts"]

In [4]:
def check_all_columns_are_constant(df,
                                   cols=[
                                       "OrganizationType",
                                       "OrganizationCode",
                                       "index"
                                   ]):
    tmpdf = df.copy()
    
    for col in cols:
        for c in tmpdf["City"].unique():
            if not column_is_constant(tmpdf.set_index(["City","report_year"]).loc[c][col]):
                print ("%s\t%s"%(col,c) )
check_all_columns_are_constant(pubs)

OrganizationCode	DEEP RIVER
OrganizationCode	EASTON
index	DEEP RIVER
index	EASTON


In [7]:
import numpy as np
def test_sum(series):
    
    return reduce(lambda x, y: x + y, series)

print test_sum([1,2,3,4])
print test_sum([1,2,3,np.nan])
test_sum_df = pubs.groupby(["City","report_year"]).agg({'Education spending': ['sum', test_sum]})
#print "Columns match: %d" % test_sum_df[test_sum_df[("Education spending","sum")] != test_sum_df[("Education spending","test_sum")]]


10
nan


In [ ]:
print "Some FTE counts are zero"
pubs[pubs["FTE Count"] == 0]

In [ ]:
print "Some categories are null"
print "Somet of these nulls are legit and others aren't"
find_nulls = pd.pivot_table(pubs,
               index=["City","report_year"],
               values="FTE Count",
               columns=["Assignment Category",
                        "Educator Type"])
any_nulls = find_nulls[find_nulls.isnull().any(axis=1)]\

any_nulls.to_csv("output/any-nulls.csv")

any_nulls.head()

In [ ]:
print "All cities with aany nulls"
any_nulls.reset_index()["City"].unique()

In [ ]:
any_nulls

In [ ]:
print "The only category without any nulls in gen ed teacers"
find_nulls.dropna(axis=1).head()

In [ ]:
# Get totals
print "Keep in mind, some of these have nulls. Check those before reaching any conclusions"
fte_totals = pubs.groupby(["City","report_year"]).agg(
    {"FTE Count":sum,
     "Education spending":max,
     "Enrollment":max
})

#fte_totals.to_csv("output/total-fte.csv")

# Add true false if the index has nulls
fte_totals.head()

In [ ]:
def has_nulls((city,year)):
    return (city, year) in any_nulls.index

print has_nulls(("Norwalk", 2010))
print has_nulls(("NORWALK", 2010))

In [ ]:
fte_totals["has_nulls"] = fte_totals.index.map(has_nulls)
len(fte_totals)

In [ ]:
len(fte_totals[fte_totals["has_nulls"]])

In [ ]:
len(fte_totals[fte_totals["has_nulls"] == False ])

In [ ]:
fte_totals.to_csv("output/total-fte.csv")